In [12]:
import os
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, roc_auc_score, classification_report
import os
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import os 
import pandas as pd
import numpy as np
import random
import pickle
import pandas as pd
from Bio import SeqIO
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Input, Flatten, LSTM, Dropout, Bidirectional, LeakyReLU, Reshape, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score
# performance matrices
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score

# plots
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from numpy import array
from numpy import argmax

from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import roc_curve

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.models import load_model
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import imblearn
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, roc_auc_score, classification_report
from imblearn.under_sampling import RandomUnderSampler

In [2]:
os.chdir("/homes/t326h379/Ankh_nucleocytoplasmic_dbptm_2231")
df_positive = pd.read_csv("Ankh_Feature_5724_Positive_Taining_Site_Intracellular_from_1638_Proteins.txt",header=None)
df_negative = pd.read_csv("Ankh_Feature_232286_Negative_Taining_Site_Intracellular_from_1638_Proteins.txt",header = None)

df_positive_positive = df_positive
df_negative_negative = df_negative

Header_name = ["Position","PID","S_or_T"]+[i for i in range(1,1537)]
df_positive_positive.columns = Header_name
df_negative_negative.columns = Header_name

df_positive.columns = Header_name
df_negative.columns = Header_name

In [7]:
List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))

In [8]:
len(List_of_Training_Protein)

1638

In [9]:
df_positive_positive.head()

,Position,PID,S_or_T,1,2,3,4,5,6,7,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,1536
0,387,Q9UQE7,S,-0.006849,0.016360,0.025910,0.030055,-0.067093,0.019464,0.036232,...,-0.003482,-0.027595,0.033556,0.003565,-0.012091,0.016382,0.006348,-0.017497,-0.024048,0.017180
1,817,Q8IXF0,S,-0.008763,-0.006594,0.000093,0.014235,0.029058,0.013764,-0.011274,...,-0.007867,0.013020,-0.007994,-0.001646,-0.023366,-0.014445,0.014405,0.002329,0.008613,-0.004797
2,348,Q8N684,S,0.033468,0.012824,-0.011073,-0.000779,-0.016336,-0.010204,-0.029563,...,0.020125,0.008858,0.003539,0.004413,-0.020244,-0.013336,0.006468,-0.012328,0.012469,0.016119
3,2682,Q9Y6V0,T,0.039638,0.013787,0.019586,0.009582,0.040503,-0.028107,0.001097,...,-0.001421,-0.025978,0.010091,-0.001156,-0.017147,-0.020890,0.015685,0.006286,-0.002682,0.029310
4,2093,Q14980,T,0.026931,0.010902,0.016720,0.012042,0.031143,-0.020605,0.004498,...,-0.010224,-0.000175,-0.020566,-0.005050,-0.008972,-0.010151,0.049726,-0.016886,0.001899,0.008699


In [24]:
a = random.sample(range(1, 1000000), 40)

for i in range(len(a)):
    seed = a[i]

    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)

    windows = []
    start = 0
    stride = 163
    window_size = 163
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_Ankh = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_Ankh  = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_Ankh  = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_Ankh  = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)

        print("Training Dataset")
        print(X_train_full_Ankh.shape ,y_train_full_Ankh.shape)
        X_train, y_train = rus.fit_resample(X_train_full_Ankh ,y_train_full_Ankh)
        print(X_train.shape,y_train.shape)

       
        X_train_train, X_test_test, Y_train_train, Y_test_test = train_test_split(X_train, y_train, test_size = 0.1, random_state = seed)

        y_train_1_1 = tf.keras.utils.to_categorical(Y_train_train,2)
        y_val_1_1 = tf.keras.utils.to_categorical(Y_test_test,2)

        print("Validation Dataset")
        print(X_val_full_Ankh.shape,y_val_full_Ankh.shape)

        print("\n\n\n")
        
        model = Sequential()

        model.add(Input(shape=(1536,)))
        
        model.add(Dense(512,activation='relu'))
        model.add(Dropout(0.3))
        
        model.add(Dense(256,activation='relu'))
        model.add(Dropout(0.3))

        model.add(Dense(32,activation='relu'))
        model.add(Dropout(0.3))

        model.add(Dense(2,activation='softmax'))

        model.compile(optimizer=tf.keras.optimizers.Adam(),loss="binary_crossentropy",metrics=["accuracy"])


        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="ROC_ROC_Premise_Assumption.h5", 
                                        monitor = 'val_accuracy',
                                        verbose=0, 
                                        save_weights_only=False,
                                        save_best_only=True)

        reduce_lr_acc = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.001, patience=5, verbose=1, min_delta=1e-4, mode='max')

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max')

        history = model.fit(X_train_train, y_train_1_1,epochs=6,verbose=1,batch_size=256,
                                callbacks=[checkpointer,reduce_lr_acc, early_stopping],validation_data=(X_test_test, y_val_1_1))
        
        
        xval,yval = rus.fit_resample(X_val_full_Ankh, y_val_full_Ankh)
#         xval = X_val_full_ProtT5
#         yval = y_val_full_ProtT5
        print(xval.shape,yval.shape)
        
        Y_pred = model.predict(xval)
        Y_pred = (Y_pred > 0.5)
        y_pred = [np.argmax(y, axis=None, out=None) for y in Y_pred]
        y_pred = np.array(y_pred)

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("MLP Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(5064, 1536) (640, 1536)
(209680, 1536) (21695, 1536)
Training Dataset
(214744, 1536) (214744,)
(10128, 1536) (10128,)
Validation Dataset
(22335, 1536) (22335,)




Epoch 1/6
36/36 [==============================] - 2s 23ms/step - loss: 0.6406 - accuracy: 0.6560 - val_loss: 0.5850 - val_accuracy: 0.6851 - lr: 0.0010
Epoch 2/6
36/36 [==============================] - 1s 16ms/step - loss: 0.5624 - accuracy: 0.7196 - val_loss: 0.5711 - val_accuracy: 0.7009 - lr: 0.0010
Epoch 3/6
36/36 [==============================] - 1s 15ms/step - loss: 0.5286 - accuracy: 0.7462 - val_loss: 0.5700 - val_accuracy: 0.7019 - lr: 0.0010
Epoch 4/6
36/36 [==============================] - 0s 10ms/step - loss: 0.4895 - accuracy: 0.7751 - val_loss: 0.5944 - val_accuracy: 0.6802 - lr: 0.0010
Epoch 5/6
36/36 [==============================] - 0s 9ms/step - loss: 0.4513 - accuracy: 0.7970 - val_loss: 0.6039 - val_accuracy: 0.6940 - lr: 0.0010
Epoch 6/6
36/36 [==============================] - 0s 10ms/step - loss:

Epoch 6/6
37/37 [==============================] - 0s 10ms/step - loss: 0.4019 - accuracy: 0.8253 - val_loss: 0.6057 - val_accuracy: 0.7071 - lr: 0.0010
(1036, 1536) (1036,)
33/33 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.4324627348489404
Confusion Matrix : 
 [[423  95]
 [204 314]]
Accuracy on test set:    0.7113899613899614
Sensitivity:    0.6061776061776062 	 Specificity:    0.8166023166023166
Precision  : 0.7677261613691931
Area Under Curve:    0.7113899613899614
(1036, 1536) (1036,)
(5112, 1536) (592, 1536)
(209926, 1536) (21449, 1536)
Training Dataset
(215038, 1536) (215038,)
(10224, 1536) (10224,)
Validation Dataset
(22041, 1536) (22041,)




Epoch 1/6
36/36 [==============================] - 2s 23ms/step - loss: 0.6369 - accuracy: 0.6553 - val_loss: 0.5763 - val_accuracy: 0.7028 - lr: 0.0010
Epoch 2/6
36/36 [==============================] - 1s 16ms/step - loss: 0.5627 - accuracy: 0.7224 - val_loss: 0.5563 - val_accuracy: 0.7097 

Epoch 1/6
36/36 [==============================] - 5s 114ms/step - loss: 0.6277 - accuracy: 0.6679 - val_loss: 0.5623 - val_accuracy: 0.7209 - lr: 0.0010
Epoch 2/6
36/36 [==============================] - 4s 121ms/step - loss: 0.5504 - accuracy: 0.7296 - val_loss: 0.5493 - val_accuracy: 0.7229 - lr: 0.0010
Epoch 3/6
36/36 [==============================] - 10s 280ms/step - loss: 0.5206 - accuracy: 0.7496 - val_loss: 0.5435 - val_accuracy: 0.7239 - lr: 0.0010
Epoch 4/6
36/36 [==============================] - 4s 121ms/step - loss: 0.4907 - accuracy: 0.7699 - val_loss: 0.5450 - val_accuracy: 0.7249 - lr: 0.0010
Epoch 5/6
36/36 [==============================] - 0s 10ms/step - loss: 0.4455 - accuracy: 0.8028 - val_loss: 0.5758 - val_accuracy: 0.7219 - lr: 0.0010
Epoch 6/6
36/36 [==============================] - 0s 10ms/step - loss: 0.3884 - accuracy: 0.8353 - val_loss: 0.6350 - val_accuracy: 0.7150 - lr: 0.0010
(1276, 1536) (1276,)
40/40 [==============================] - 0s 1ms/step
Ind

35/35 [==============================] - 0s 10ms/step - loss: 0.4109 - accuracy: 0.8248 - val_loss: 0.6010 - val_accuracy: 0.6934 - lr: 0.0010
(1564, 1536) (1564,)
49/49 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.4365503007361343
Confusion Matrix : 
 [[580 202]
 [239 543]]
Accuracy on test set:    0.7180306905370843
Sensitivity:    0.6943734015345269 	 Specificity:    0.7416879795396419
Precision  : 0.7288590604026846
Area Under Curve:    0.7180306905370843
(1564, 1536) (1564,)
(5256, 1536) (450, 1536)
(208198, 1536) (22191, 1536)
Training Dataset
(213454, 1536) (213454,)
(10512, 1536) (10512,)
Validation Dataset
(22641, 1536) (22641,)




Epoch 1/6
37/37 [==============================] - 7s 159ms/step - loss: 0.6252 - accuracy: 0.6621 - val_loss: 0.5606 - val_accuracy: 0.7072 - lr: 0.0010
Epoch 2/6
37/37 [==============================] - 8s 208ms/step - loss: 0.5517 - accuracy: 0.7314 - val_loss: 0.5488 - val_accuracy: 0.7148 - lr: 0.

Epoch 1/6
36/36 [==============================] - 2s 21ms/step - loss: 0.6300 - accuracy: 0.6502 - val_loss: 0.5838 - val_accuracy: 0.6824 - lr: 0.0010
Epoch 2/6
36/36 [==============================] - 1s 17ms/step - loss: 0.5551 - accuracy: 0.7243 - val_loss: 0.5643 - val_accuracy: 0.7039 - lr: 0.0010
Epoch 3/6
36/36 [==============================] - 0s 10ms/step - loss: 0.5280 - accuracy: 0.7464 - val_loss: 0.5611 - val_accuracy: 0.7039 - lr: 0.0010
Epoch 4/6
36/36 [==============================] - 0s 9ms/step - loss: 0.4989 - accuracy: 0.7670 - val_loss: 0.5760 - val_accuracy: 0.7020 - lr: 0.0010
Epoch 5/6
36/36 [==============================] - 1s 14ms/step - loss: 0.4595 - accuracy: 0.7929 - val_loss: 0.5802 - val_accuracy: 0.7108 - lr: 0.0010
Epoch 6/6
36/36 [==============================] - 0s 10ms/step - loss: 0.4045 - accuracy: 0.8221 - val_loss: 0.6028 - val_accuracy: 0.7088 - lr: 0.0010
(1208, 1536) (1208,)
38/38 [==============================] - 0s 2ms/step
Independe

...

(5177, 1536) (534, 1536)
(209487, 1536) (21454, 1536)
Training Dataset
(214664, 1536) (214664,)
(10354, 1536) (10354,)
Validation Dataset
(21988, 1536) (21988,)




Epoch 1/6
37/37 [==============================] - 2s 27ms/step - loss: 0.6276 - accuracy: 0.6586 - val_loss: 0.5653 - val_accuracy: 0.7056 - lr: 0.0010
Epoch 2/6
37/37 [==============================] - 1s 17ms/step - loss: 0.5526 - accuracy: 0.7269 - val_loss: 0.5484 - val_accuracy: 0.7220 - lr: 0.0010
Epoch 3/6
37/37 [==============================] - 0s 10ms/step - loss: 0.5236 - accuracy: 0.7476 - val_loss: 0.5683 - val_accuracy: 0.7133 - lr: 0.0010
Epoch 4/6
37/37 [==============================] - 1s 15ms/step - loss: 0.5024 - accuracy: 0.7652 - val_loss: 0.5425 - val_accuracy: 0.7346 - lr: 0.0010
Epoch 5/6
37/37 [==============================] - 0s 10ms/step - loss: 0.4719 - accuracy: 0.7848 - val_loss: 0.5510 - val_accuracy: 0.7239 - lr: 0.0010
Epoch 6/6
37/37 [==============================] - 0s 10ms/step - loss

37/37 [==============================] - 0s 9ms/step - loss: 0.4558 - accuracy: 0.7973 - val_loss: 0.5206 - val_accuracy: 0.7536 - lr: 0.0010
Epoch 6/6
37/37 [==============================] - 1s 15ms/step - loss: 0.4060 - accuracy: 0.8266 - val_loss: 0.5380 - val_accuracy: 0.7643 - lr: 0.0010
(1074, 1536) (1074,)
34/34 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.3387910128561248
Confusion Matrix : 
 [[406 131]
 [227 310]]
Accuracy on test set:    0.6666666666666666
Sensitivity:    0.5772811918063314 	 Specificity:    0.7560521415270018
Precision  : 0.7029478458049887
Area Under Curve:    0.6666666666666665
(1074, 1536) (1074,)
(5117, 1536) (594, 1536)
(208303, 1536) (22638, 1536)
Training Dataset
(213420, 1536) (213420,)
(10234, 1536) (10234,)
Validation Dataset
(23232, 1536) (23232,)




Epoch 1/6
36/36 [==============================] - 2s 21ms/step - loss: 0.6365 - accuracy: 0.6372 - val_loss: 0.5914 - val_accuracy: 0.6943 - lr: 0.001

(5121, 1536) (587, 1536)
(207055, 1536) (24129, 1536)
Training Dataset
(212176, 1536) (212176,)
(10242, 1536) (10242,)
Validation Dataset
(24716, 1536) (24716,)




Epoch 1/6
37/37 [==============================] - 2s 22ms/step - loss: 0.6425 - accuracy: 0.6521 - val_loss: 0.5723 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 2/6
37/37 [==============================] - 1s 14ms/step - loss: 0.5581 - accuracy: 0.7280 - val_loss: 0.5571 - val_accuracy: 0.7093 - lr: 0.0010
Epoch 3/6
37/37 [==============================] - 1s 14ms/step - loss: 0.5299 - accuracy: 0.7520 - val_loss: 0.5509 - val_accuracy: 0.7249 - lr: 0.0010
Epoch 4/6
37/37 [==============================] - 0s 9ms/step - loss: 0.5041 - accuracy: 0.7675 - val_loss: 0.5482 - val_accuracy: 0.7122 - lr: 0.0010
Epoch 5/6
37/37 [==============================] - 0s 9ms/step - loss: 0.4861 - accuracy: 0.7834 - val_loss: 0.5636 - val_accuracy: 0.7132 - lr: 0.0010
Epoch 6/6
37/37 [==============================] - 0s 9ms/step - loss: 0

37/37 [==============================] - 10s 278ms/step - loss: 0.5042 - accuracy: 0.7568 - val_loss: 0.5670 - val_accuracy: 0.7103 - lr: 0.0010
Epoch 5/6
37/37 [==============================] - 0s 9ms/step - loss: 0.4604 - accuracy: 0.7882 - val_loss: 0.5817 - val_accuracy: 0.6899 - lr: 0.0010
Epoch 6/6
37/37 [==============================] - 0s 9ms/step - loss: 0.4151 - accuracy: 0.8146 - val_loss: 0.6285 - val_accuracy: 0.7035 - lr: 0.0010
(1096, 1536) (1096,)
35/35 [==============================] - 0s 1ms/step
Independent Testing Matthews Correlation:  0.4420046909838443
Confusion Matrix : 
 [[357 191]
 [117 431]]
Accuracy on test set:    0.718978102189781
Sensitivity:    0.7864963503649635 	 Specificity:    0.6514598540145985
Precision  : 0.6929260450160771
Area Under Curve:    0.718978102189781
(1096, 1536) (1096,)
MLP Mean MCC = 0.3809 + 0.0591 and Mean AUC = 0.6885 + 0.0304
Mean Sensitivity = 0.6699 + 0.0889 and Mean Specificity = 0.7071 + 0.0642 and Mean Accuracy = 0.6885 +

30/30 [==============================] - 0s 2ms/step
Independent Testing Matthews Correlation:  0.34561852290353995
Confusion Matrix : 
 [[337 131]
 [176 292]]
Accuracy on test set:    0.6720085470085471
Sensitivity:    0.6239316239316239 	 Specificity:    0.7200854700854701
Precision  : 0.6903073286052009
Area Under Curve:    0.672008547008547
(936, 1536) (936,)
(5151, 1536) (553, 1536)
(210069, 1536) (21543, 1536)
Training Dataset
(215220, 1536) (215220,)
(10302, 1536) (10302,)
Validation Dataset
(22096, 1536) (22096,)




Epoch 1/6
37/37 [==============================] - 3s 46ms/step - loss: 0.6212 - accuracy: 0.6658 - val_loss: 0.5583 - val_accuracy: 0.7187 - lr: 0.0010
Epoch 2/6
37/37 [==============================] - 7s 183ms/step - loss: 0.5545 - accuracy: 0.7256 - val_loss: 0.5301 - val_accuracy: 0.7498 - lr: 0.0010
Epoch 3/6
37/37 [==============================] - 0s 10ms/step - loss: 0.5231 - accuracy: 0.7506 - val_loss: 0.5373 - val_accuracy: 0.7333 - lr: 0.0010
Epoch 4/6

# Logistic Regression 10 Fold CV

In [17]:
a = random.sample(range(1, 1000000), 10)
from sklearn.linear_model import LogisticRegression

for i in range(len(a)):
    seed = a[i]

    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)

    windows = []
    start = 0
    stride = 163
    window_size = 163
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_Ankh = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_Ankh  = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_Ankh  = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_Ankh  = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)
        
        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_Ankh,y_train_full_Ankh)
        print(X_train.shape,y_train.shape)

        print("\n\n\n")

        clf = LogisticRegression(penalty='l2',solver='lbfgs',random_state=seed)

        clf.fit(X_train, y_train)
        print("Validation Dataset")
        
        xval,yval = rus.fit_resample(X_val_full_Ankh,y_val_full_Ankh)
        print(xval.shape,yval.shape)

        print(xval.shape,yval.shape)   
        
        y_pred = np.array(clf.predict(xval))

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("LR Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(5146, 1536) (552, 1536)
(206789, 1536) (24387, 1536)
Training Dataset
(10292, 1536) (10292,)




Validation Dataset
(1104, 1536) (1104,)
(1104, 1536) (1104,)
Independent Testing Matthews Correlation:  0.39604409287959536
Confusion Matrix : 
 [[418 134]
 [201 351]]
Accuracy on test set:    0.6965579710144928
Sensitivity:    0.6358695652173914 	 Specificity:    0.7572463768115942
Precision  : 0.7237113402061855
Area Under Curve:    0.6965579710144928
(1104, 1536) (1104,)
(5076, 1536) (622, 1536)
(206545, 1536) (24631, 1536)
Training Dataset
(10152, 1536) (10152,)




Validation Dataset
(1244, 1536) (1244,)
(1244, 1536) (1244,)
Independent Testing Matthews Correlation:  0.4474462603796272
Confusion Matrix : 
 [[489 133]
 [213 409]]
Accuracy on test set:    0.7218649517684887
Sensitivity:    0.657556270096463 	 Specificity:    0.7861736334405145
Precision  : 0.7546125461254612
Area Under Curve:    0.7218649517684887
(1244, 1536) (1244,)
(5034, 1536) (664, 1536)
(206999, 1536) (24177, 1536

(4976, 1536) (723, 1536)
(205709, 1536) (25047, 1536)
Training Dataset
(9952, 1536) (9952,)




Validation Dataset
(1446, 1536) (1446,)
(1446, 1536) (1446,)
Independent Testing Matthews Correlation:  0.522015981964714
Confusion Matrix : 
 [[567 156]
 [190 533]]
Accuracy on test set:    0.7607192254495159
Sensitivity:    0.7372060857538036 	 Specificity:    0.7842323651452282
Precision  : 0.7735849056603774
Area Under Curve:    0.760719225449516
(1446, 1536) (1446,)
LR Mean MCC = 0.4264 + 0.0646 and Mean AUC = 0.7114 + 0.0331
Mean Sensitivity = 0.6555 + 0.0666 and Mean Specificity = 0.7673 + 0.0265 and Mean Accuracy = 0.7114 + 0.0331
Mean Precision = 0.7375 + 0.0262





(5213, 1536) (495, 1536)
(208792, 1536) (21959, 1536)
Training Dataset
(10426, 1536) (10426,)




Validation Dataset
(990, 1536) (990,)
(990, 1536) (990,)
Independent Testing Matthews Correlation:  0.4322345940893118
Confusion Matrix : 
 [[394 101]
 [183 312]]
Accuracy on test set:    0.7131313131313132
Sensitivity:    

(5145, 1536) (567, 1536)
(208699, 1536) (22632, 1536)
Training Dataset
(10290, 1536) (10290,)




Validation Dataset
(1134, 1536) (1134,)
(1134, 1536) (1134,)
Independent Testing Matthews Correlation:  0.4526140999202622
Confusion Matrix : 
 [[451 116]
 [197 370]]
Accuracy on test set:    0.7239858906525574
Sensitivity:    0.6525573192239859 	 Specificity:    0.7954144620811288
Precision  : 0.7613168724279835
Area Under Curve:    0.7239858906525574
(1134, 1536) (1134,)
(4959, 1536) (753, 1536)
(206655, 1536) (24676, 1536)
Training Dataset
(9918, 1536) (9918,)




Validation Dataset
(1506, 1536) (1506,)
(1506, 1536) (1506,)
Independent Testing Matthews Correlation:  0.518005489729346
Confusion Matrix : 
 [[565 188]
 [175 578]]
Accuracy on test set:    0.7589641434262948
Sensitivity:    0.7675962815405046 	 Specificity:    0.750332005312085
Precision  : 0.7545691906005222
Area Under Curve:    0.7589641434262948
(1506, 1536) (1506,)
LR Mean MCC = 0.4239 + 0.0744 and Mean AUC = 0.7100 + 0.

(5218, 1536) (462, 1536)
(207531, 1536) (23964, 1536)
Training Dataset
(10436, 1536) (10436,)




Validation Dataset
(924, 1536) (924,)
(924, 1536) (924,)
Independent Testing Matthews Correlation:  0.3580103399607398
Confusion Matrix : 
 [[343 119]
 [179 283]]
Accuracy on test set:    0.6774891774891775
Sensitivity:    0.6125541125541125 	 Specificity:    0.7424242424242424
Precision  : 0.7039800995024875
Area Under Curve:    0.6774891774891775
(924, 1536) (924,)
(4973, 1536) (707, 1536)
(207005, 1536) (24490, 1536)
Training Dataset
(9946, 1536) (9946,)




Validation Dataset
(1414, 1536) (1414,)
(1414, 1536) (1414,)
Independent Testing Matthews Correlation:  0.5218687554129977
Confusion Matrix : 
 [[563 144]
 [195 512]]
Accuracy on test set:    0.7602545968882602
Sensitivity:    0.7241867043847242 	 Specificity:    0.7963224893917963
Precision  : 0.7804878048780488
Area Under Curve:    0.7602545968882604
(1414, 1536) (1414,)
(5141, 1536) (539, 1536)
(205402, 1536) (26093, 1536)
Traini

(5171, 1536) (541, 1536)
(207458, 1536) (23536, 1536)
Training Dataset
(10342, 1536) (10342,)




Validation Dataset
(1082, 1536) (1082,)
(1082, 1536) (1082,)
Independent Testing Matthews Correlation:  0.42804428748788365
Confusion Matrix : 
 [[405 136]
 [174 367]]
Accuracy on test set:    0.7134935304990758
Sensitivity:    0.678373382624769 	 Specificity:    0.7486136783733827
Precision  : 0.7296222664015904
Area Under Curve:    0.7134935304990757
(1082, 1536) (1082,)
(5194, 1536) (518, 1536)
(207286, 1536) (23708, 1536)
Training Dataset
(10388, 1536) (10388,)




Validation Dataset
(1036, 1536) (1036,)
(1036, 1536) (1036,)
Independent Testing Matthews Correlation:  0.42049089412267476
Confusion Matrix : 
 [[408 110]
 [193 325]]
Accuracy on test set:    0.7075289575289575
Sensitivity:    0.6274131274131274 	 Specificity:    0.7876447876447876
Precision  : 0.7471264367816092
Area Under Curve:    0.7075289575289575
(1036, 1536) (1036,)
(5266, 1536) (446, 1536)
(208020, 1536) (22974, 153

(5101, 1536) (600, 1536)
(208715, 1536) (22147, 1536)
Training Dataset
(10202, 1536) (10202,)




Validation Dataset
(1200, 1536) (1200,)
(1200, 1536) (1200,)
Independent Testing Matthews Correlation:  0.4228212260749882
Confusion Matrix : 
 [[476 124]
 [226 374]]
Accuracy on test set:    0.7083333333333334
Sensitivity:    0.6233333333333333 	 Specificity:    0.7933333333333333
Precision  : 0.751004016064257
Area Under Curve:    0.7083333333333333
(1200, 1536) (1200,)
(5083, 1536) (618, 1536)
(206230, 1536) (24632, 1536)
Training Dataset
(10166, 1536) (10166,)




Validation Dataset
(1236, 1536) (1236,)
(1236, 1536) (1236,)
Independent Testing Matthews Correlation:  0.39601832624924266
Confusion Matrix : 
 [[455 163]
 [211 407]]
Accuracy on test set:    0.6974110032362459
Sensitivity:    0.6585760517799353 	 Specificity:    0.7362459546925566
Precision  : 0.7140350877192982
Area Under Curve:    0.6974110032362459
(1236, 1536) (1236,)
(4981, 1536) (720, 1536)
(209834, 1536) (21028, 1536

# Random Forest 10 Fold CV

In [18]:
a = random.sample(range(1, 1000000), 10)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


for i in range(len(a)):
    seed = a[i]

    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)

    windows = []
    start = 0
    stride = 163
    window_size = 163
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_Ankh = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_Ankh  = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_Ankh  = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_Ankh  = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)
        
        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_Ankh,y_train_full_Ankh)
        print(X_train.shape,y_train.shape)

        print("\n\n\n")

        clf = RandomForestClassifier(max_depth=10, random_state=seed)

        clf.fit(X_train, y_train)
        print("Validation Dataset")
        
        xval,yval = rus.fit_resample(X_val_full_Ankh,y_val_full_Ankh)
        print(xval.shape,yval.shape)

        print(xval.shape,yval.shape)   
        
        y_pred = np.array(clf.predict(xval))

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("RF Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(5021, 1536) (689, 1536)
(205873, 1536) (25605, 1536)
Training Dataset
(10042, 1536) (10042,)




Validation Dataset
(1378, 1536) (1378,)
(1378, 1536) (1378,)
Independent Testing Matthews Correlation:  0.4714742500304917
Confusion Matrix : 
 [[551 138]
 [229 460]]
Accuracy on test set:    0.7336719883889695
Sensitivity:    0.6676342525399129 	 Specificity:    0.7997097242380261
Precision  : 0.7692307692307693
Area Under Curve:    0.7336719883889696
(1378, 1536) (1378,)
(5101, 1536) (609, 1536)
(209262, 1536) (22216, 1536)
Training Dataset
(10202, 1536) (10202,)




Validation Dataset
(1218, 1536) (1218,)
(1218, 1536) (1218,)
Independent Testing Matthews Correlation:  0.4100857576843898
Confusion Matrix : 
 [[473 136]
 [226 383]]
Accuracy on test set:    0.7027914614121511
Sensitivity:    0.6288998357963875 	 Specificity:    0.7766830870279147
Precision  : 0.7379576107899807
Area Under Curve:    0.7027914614121511
(1218, 1536) (1218,)
(5090, 1536) (620, 1536)
(208772, 1536) (22706, 1536

(5200, 1536) (492, 1536)
(207104, 1536) (23613, 1536)
Training Dataset
(10400, 1536) (10400,)




Validation Dataset
(984, 1536) (984,)
(984, 1536) (984,)
Independent Testing Matthews Correlation:  0.34140158805027376
Confusion Matrix : 
 [[392 100]
 [230 262]]
Accuracy on test set:    0.6646341463414634
Sensitivity:    0.532520325203252 	 Specificity:    0.7967479674796748
Precision  : 0.7237569060773481
Area Under Curve:    0.6646341463414633
(984, 1536) (984,)
RF Mean MCC = 0.3884 + 0.0506 and Mean AUC = 0.6903 + 0.0267
Mean Sensitivity = 0.5911 + 0.0528 and Mean Specificity = 0.7894 + 0.0163 and Mean Accuracy = 0.6903 + 0.0267
Mean Precision = 0.7366 + 0.0206





(4814, 1536) (901, 1536)
(208104, 1536) (23360, 1536)
Training Dataset
(9628, 1536) (9628,)




Validation Dataset
(1802, 1536) (1802,)
(1802, 1536) (1802,)
Independent Testing Matthews Correlation:  0.516461079563166
Confusion Matrix : 
 [[700 201]
 [235 666]]
Accuracy on test set:    0.758046614872364
Sensitivity:    0.

(5233, 1536) (474, 1536)
(210790, 1536) (20530, 1536)
Training Dataset
(10466, 1536) (10466,)




Validation Dataset
(948, 1536) (948,)
(948, 1536) (948,)
Independent Testing Matthews Correlation:  0.39644736499293204
Confusion Matrix : 
 [[387  87]
 [205 269]]
Accuracy on test set:    0.6919831223628692
Sensitivity:    0.5675105485232067 	 Specificity:    0.8164556962025317
Precision  : 0.7556179775280899
Area Under Curve:    0.6919831223628693
(948, 1536) (948,)
(4970, 1536) (737, 1536)
(207654, 1536) (23666, 1536)
Training Dataset
(9940, 1536) (9940,)




Validation Dataset
(1474, 1536) (1474,)
(1474, 1536) (1474,)
Independent Testing Matthews Correlation:  0.43510887603485343
Confusion Matrix : 
 [[575 162]
 [257 480]]
Accuracy on test set:    0.7157394843962008
Sensitivity:    0.6512890094979648 	 Specificity:    0.7801899592944369
Precision  : 0.7476635514018691
Area Under Curve:    0.7157394843962008
(1474, 1536) (1474,)
RF Mean MCC = 0.3917 + 0.0615 and Mean AUC = 0.6916 + 0.03

(5288, 1536) (412, 1536)
(209718, 1536) (21459, 1536)
Training Dataset
(10576, 1536) (10576,)




Validation Dataset
(824, 1536) (824,)
(824, 1536) (824,)
Independent Testing Matthews Correlation:  0.24540103733372243
Confusion Matrix : 
 [[324  88]
 [229 183]]
Accuracy on test set:    0.6152912621359223
Sensitivity:    0.4441747572815534 	 Specificity:    0.7864077669902912
Precision  : 0.6752767527675276
Area Under Curve:    0.6152912621359222
(824, 1536) (824,)
(4901, 1536) (799, 1536)
(208063, 1536) (23114, 1536)
Training Dataset
(9802, 1536) (9802,)




Validation Dataset
(1598, 1536) (1598,)
(1598, 1536) (1598,)
Independent Testing Matthews Correlation:  0.5306836956709631
Confusion Matrix : 
 [[608 191]
 [184 615]]
Accuracy on test set:    0.7653316645807259
Sensitivity:    0.769712140175219 	 Specificity:    0.7609511889862328
Precision  : 0.7630272952853598
Area Under Curve:    0.765331664580726
(1598, 1536) (1598,)
(5232, 1536) (468, 1536)
(209857, 1536) (21320, 1536)
Trainin

KeyboardInterrupt: 

# XGBoost 10 Fold Cross Validation

In [25]:
a = random.sample(range(1, 1000000), 10)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


for i in range(len(a)):
    seed = a[i]

    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)

    windows = []
    start = 0
    stride = 163
    window_size = 163
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_Ankh = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_Ankh  = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_Ankh  = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_Ankh  = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)
        
        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_Ankh,y_train_full_Ankh)
        print(X_train.shape,y_train.shape)

        print("\n\n\n")

        clf = XGBClassifier()
        clf.fit(X_train, y_train)
        print("Validation Dataset")
        
        xval,yval = rus.fit_resample(X_val_full_Ankh,y_val_full_Ankh)
        print(xval.shape,yval.shape)

        
        y_pred = np.array(clf.predict(xval))

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("XGBoost Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(5128, 1536) (583, 1536)
(206760, 1536) (24673, 1536)
Training Dataset
(10256, 1536) (10256,)




Validation Dataset
(1166, 1536) (1166,)
Independent Testing Matthews Correlation:  0.4088116209396277
Confusion Matrix : 
 [[426 157]
 [188 395]]
Accuracy on test set:    0.7041166380789022
Sensitivity:    0.6775300171526587 	 Specificity:    0.7307032590051458
Precision  : 0.7155797101449275
Area Under Curve:    0.7041166380789023
(1166, 1536) (1166,)
(4919, 1536) (792, 1536)
(208503, 1536) (22930, 1536)
Training Dataset
(9838, 1536) (9838,)




Validation Dataset
(1584, 1536) (1584,)
Independent Testing Matthews Correlation:  0.44073561882119283
Confusion Matrix : 
 [[578 214]
 [229 563]]
Accuracy on test set:    0.7203282828282829
Sensitivity:    0.7108585858585859 	 Specificity:    0.7297979797979798
Precision  : 0.7245817245817245
Area Under Curve:    0.7203282828282828
(1584, 1536) (1584,)
(5057, 1536) (654, 1536)
(206173, 1536) (25260, 1536)
Training Dataset
(10114, 1536) (10114,)



(5001, 1536) (692, 1536)
(206987, 1536) (24291, 1536)
Training Dataset
(10002, 1536) (10002,)




Validation Dataset
(1384, 1536) (1384,)
Independent Testing Matthews Correlation:  0.4495576424276651
Confusion Matrix : 
 [[510 182]
 [199 493]]
Accuracy on test set:    0.7247109826589595
Sensitivity:    0.7124277456647399 	 Specificity:    0.7369942196531792
Precision  : 0.7303703703703703
Area Under Curve:    0.7247109826589595
(1384, 1536) (1384,)
(5115, 1536) (578, 1536)
(206433, 1536) (24845, 1536)
Training Dataset
(10230, 1536) (10230,)




Validation Dataset
(1156, 1536) (1156,)
Independent Testing Matthews Correlation:  0.28454312147948563
Confusion Matrix : 
 [[409 169]
 [246 332]]
Accuracy on test set:    0.6410034602076125
Sensitivity:    0.5743944636678201 	 Specificity:    0.7076124567474048
Precision  : 0.6626746506986028
Area Under Curve:    0.6410034602076126
(1156, 1536) (1156,)
(4990, 1536) (703, 1536)
(210251, 1536) (21027, 1536)
Training Dataset
(9980, 1536) (9980,)



(5130, 1536) (563, 1536)
(206493, 1536) (25109, 1536)
Training Dataset
(10260, 1536) (10260,)




Validation Dataset
(1126, 1536) (1126,)
Independent Testing Matthews Correlation:  0.3271651579681946
Confusion Matrix : 
 [[405 158]
 [222 341]]
Accuracy on test set:    0.6625222024866785
Sensitivity:    0.605683836589698 	 Specificity:    0.7193605683836589
Precision  : 0.6833667334669339
Area Under Curve:    0.6625222024866786
(1126, 1536) (1126,)
(4908, 1536) (785, 1536)
(207113, 1536) (24489, 1536)
Training Dataset
(9816, 1536) (9816,)




Validation Dataset
(1570, 1536) (1570,)
Independent Testing Matthews Correlation:  0.4911493434695844
Confusion Matrix : 
 [[564 221]
 [179 606]]
Accuracy on test set:    0.7452229299363057
Sensitivity:    0.7719745222929937 	 Specificity:    0.7184713375796178
Precision  : 0.7327690447400241
Area Under Curve:    0.7452229299363058
(1570, 1536) (1570,)
(5095, 1536) (598, 1536)
(208511, 1536) (23091, 1536)
Training Dataset
(10190, 1536) (10190,)





(5052, 1536) (649, 1536)
(209306, 1536) (21805, 1536)
Training Dataset
(10104, 1536) (10104,)




Validation Dataset
(1298, 1536) (1298,)
Independent Testing Matthews Correlation:  0.43307458018232264
Confusion Matrix : 
 [[458 191]
 [177 472]]
Accuracy on test set:    0.7164869029275809
Sensitivity:    0.7272727272727273 	 Specificity:    0.7057010785824345
Precision  : 0.7119155354449472
Area Under Curve:    0.7164869029275809
(1298, 1536) (1298,)
(5153, 1536) (548, 1536)
(207832, 1536) (23279, 1536)
Training Dataset
(10306, 1536) (10306,)




Validation Dataset
(1096, 1536) (1096,)
Independent Testing Matthews Correlation:  0.43169437304747954
Confusion Matrix : 
 [[423 125]
 [188 360]]
Accuracy on test set:    0.7144160583941606
Sensitivity:    0.656934306569343 	 Specificity:    0.7718978102189781
Precision  : 0.7422680412371134
Area Under Curve:    0.7144160583941606
(1096, 1536) (1096,)
(5125, 1536) (576, 1536)
(210043, 1536) (21068, 1536)
Training Dataset
(10250, 1536) (10250,)

(5215, 1536) (469, 1536)
(206827, 1536) (23904, 1536)
Training Dataset
(10430, 1536) (10430,)




Validation Dataset
(938, 1536) (938,)
Independent Testing Matthews Correlation:  0.2907607102949926
Confusion Matrix : 
 [[345 124]
 [211 258]]
Accuracy on test set:    0.6428571428571429
Sensitivity:    0.5501066098081023 	 Specificity:    0.7356076759061834
Precision  : 0.675392670157068
Area Under Curve:    0.6428571428571428
(938, 1536) (938,)
(5155, 1536) (529, 1536)
(206899, 1536) (23832, 1536)
Training Dataset
(10310, 1536) (10310,)




Validation Dataset
(1058, 1536) (1058,)
Independent Testing Matthews Correlation:  0.3458878808082744
Confusion Matrix : 
 [[402 127]
 [222 307]]
Accuracy on test set:    0.6701323251417769
Sensitivity:    0.5803402646502835 	 Specificity:    0.7599243856332704
Precision  : 0.7073732718894009
Area Under Curve:    0.670132325141777
(1058, 1536) (1058,)
(5202, 1536) (482, 1536)
(208670, 1536) (22061, 1536)
Training Dataset
(10404, 1536) (10404,)




Va

# SVM 10 Fold CV

In [26]:
a = random.sample(range(1, 1000000), 10)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import svm

for i in range(len(a)):
    seed = a[i]

    List_of_Training_Protein = list(set(set(df_positive["PID"]).union(set(df_negative["PID"]))))
    import random
    random.seed(seed)
    random.shuffle(List_of_Training_Protein)

    windows = []
    start = 0
    stride = 163
    window_size = 163
    while start + window_size <= len(List_of_Training_Protein):
        end = start + window_size
        window = List_of_Training_Protein[start:end]
        windows.append(window)
        start += stride

    # Perform cross-validation
    cross_val_sets = []
    for i in range(9):
        validation_set = windows[i]
        training_set = [item for j, fold in enumerate(windows) if j != i for item in fold]
        cross_val_sets.append((training_set, validation_set))

    cvscores, auc_scores, sn, sp, acc, precision = list(), list(), list(), list(), list(), list()
    from sklearn.metrics import roc_curve, roc_auc_score, classification_report, auc

    for i in range(len(cross_val_sets)):
        Training_Dataset_to_Be_Trained_On = list(cross_val_sets[i][0])
        Validation_Dataset_to_Be_Validated_upon = list(cross_val_sets[i][1])
        df_positive_Training_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_positive_Validation_for_10_fold_CV = df_positive_positive[df_positive_positive['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        df_Negative_Training_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Training_Dataset_to_Be_Trained_On)]
        df_Negative_Validation_for_10_fold_CV = df_negative_negative[df_negative_negative['PID'].isin(Validation_Dataset_to_Be_Validated_upon)]

        Positive_Training_Dataset_FV = df_positive_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Positive_Validation_Dataset_FV = df_positive_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Negative_Training_Dataset_FV = df_Negative_Training_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)
        Negative_Validation_Dataset_FV = df_Negative_Validation_for_10_fold_CV.drop(["Position","PID","S_or_T"],axis=1)

        Train_Positive_Feature_Vector = np.array(Positive_Training_Dataset_FV)
        Validation_Positive_Feature_Vector = np.array(Positive_Validation_Dataset_FV)

        Train_Negative_Feature_Vector = np.array(Negative_Training_Dataset_FV)
        Validation_Negative_Feature_Vector = np.array(Negative_Validation_Dataset_FV)

        print(Train_Positive_Feature_Vector.shape, Validation_Positive_Feature_Vector.shape)
        print(Train_Negative_Feature_Vector.shape, Validation_Negative_Feature_Vector.shape)

        y_train_full_Ankh = np.array([1]*Train_Positive_Feature_Vector.shape[0]+[0]*Train_Negative_Feature_Vector.shape[0])

        y_val_full_Ankh  = np.array([1]*Validation_Positive_Feature_Vector.shape[0]+[0]*Validation_Negative_Feature_Vector.shape[0])

        X_train_full_Ankh  = np.vstack((Train_Positive_Feature_Vector,Train_Negative_Feature_Vector))

        X_val_full_Ankh  = np.vstack((Validation_Positive_Feature_Vector,Validation_Negative_Feature_Vector))

        rus = RandomUnderSampler(random_state = seed)
        
        print("Training Dataset")
        X_train, y_train = rus.fit_resample(X_train_full_Ankh,y_train_full_Ankh)
        print(X_train.shape,y_train.shape)

        print("\n\n\n")

        clf = svm.SVC(C=10, kernel='rbf')
        clf.fit(X_train, y_train)
        print("Validation Dataset")
        
        xval,yval = rus.fit_resample(X_val_full_Ankh,y_val_full_Ankh)
        print(xval.shape,yval.shape)

        
        y_pred = np.array(clf.predict(xval))

        Accurayc_laurie = accuracy_score(yval, y_pred)

        print("Independent Testing Matthews Correlation: ",matthews_corrcoef(yval, y_pred))
        print("Confusion Matrix : \n",confusion_matrix(yval, y_pred))
        print("Accuracy on test set:   ",Accurayc_laurie)

        cm = confusion_matrix(yval, y_pred)

        TP = cm[1][1]
        TN = cm[0][0]
        FP = cm[0][1]
        FN = cm[1][0]

        mcc = matthews_corrcoef(yval, y_pred)

        cvscores.append(mcc)


        Sensitivity = TP/(TP+FN)

        Specificity = TN/(TN+FP)

        sn.append(Sensitivity)
        sp.append(Specificity)

        acc.append(Accurayc_laurie)

        print("Sensitivity:   ",Sensitivity,"\t","Specificity:   ",Specificity)


        fpr, tpr, _ = roc_curve(yval, y_pred)

        roc_auc_test = auc(fpr,tpr)

        pre = TP / (TP + FP)

        print("Precision  :",pre)

        precision.append(pre)


        print("Area Under Curve:   ",roc_auc_test)

        auc_scores.append(roc_auc_test)

        print(xval.shape, yval.shape)

    print("SVM Mean MCC = %.4f + %.4f and Mean AUC = %.4f + %.4f" % (np.mean(cvscores),np.std(cvscores),np.mean(auc_scores),np.std(auc_scores)))

    print("Mean Sensitivity = %.4f + %.4f and Mean Specificity = %.4f + %.4f and Mean Accuracy = %.4f + %.4f" % (np.mean(sn),np.std(sn),np.mean(sp),np.std(sp),np.mean(acc),np.std(acc)))

    print("Mean Precision = %.4f + %.4f" % (np.mean(precision),np.std(precision)))

    print("\n\n\n\n")

(4906, 1536) (802, 1536)
(210686, 1536) (20706, 1536)
Training Dataset
(9812, 1536) (9812,)




Validation Dataset
(1604, 1536) (1604,)
Independent Testing Matthews Correlation:  0.46012872401299143
Confusion Matrix : 
 [[590 212]
 [221 581]]
Accuracy on test set:    0.7300498753117207
Sensitivity:    0.7244389027431422 	 Specificity:    0.7356608478802993
Precision  : 0.7326607818411097
Area Under Curve:    0.7300498753117207
(1604, 1536) (1604,)
(5185, 1536) (523, 1536)
(207507, 1536) (23885, 1536)
Training Dataset
(10370, 1536) (10370,)




Validation Dataset
(1046, 1536) (1046,)
Independent Testing Matthews Correlation:  0.3160257320469437
Confusion Matrix : 
 [[376 147]
 [212 311]]
Accuracy on test set:    0.6567877629063098
Sensitivity:    0.5946462715105163 	 Specificity:    0.7189292543021033
Precision  : 0.6790393013100436
Area Under Curve:    0.6567877629063098
(1046, 1536) (1046,)
(5096, 1536) (612, 1536)
(207270, 1536) (24122, 1536)
Training Dataset
(10192, 1536) (10192,)



(5164, 1536) (547, 1536)
(207477, 1536) (23713, 1536)
Training Dataset
(10328, 1536) (10328,)




Validation Dataset
(1094, 1536) (1094,)
Independent Testing Matthews Correlation:  0.35162040841167785
Confusion Matrix : 
 [[411 136]
 [221 326]]
Accuracy on test set:    0.6736745886654479
Sensitivity:    0.5959780621572212 	 Specificity:    0.7513711151736746
Precision  : 0.7056277056277056
Area Under Curve:    0.6736745886654479
(1094, 1536) (1094,)
(5231, 1536) (480, 1536)
(210116, 1536) (21074, 1536)
Training Dataset
(10462, 1536) (10462,)




Validation Dataset
(960, 1536) (960,)
Independent Testing Matthews Correlation:  0.3512537286489927
Confusion Matrix : 
 [[365 115]
 [199 281]]
Accuracy on test set:    0.6729166666666667
Sensitivity:    0.5854166666666667 	 Specificity:    0.7604166666666666
Precision  : 0.7095959595959596
Area Under Curve:    0.6729166666666667
(960, 1536) (960,)
(5034, 1536) (677, 1536)
(208896, 1536) (22294, 1536)
Training Dataset
(10068, 1536) (10068,)





(5249, 1536) (453, 1536)
(206293, 1536) (25270, 1536)
Training Dataset
(10498, 1536) (10498,)




Validation Dataset
(906, 1536) (906,)
Independent Testing Matthews Correlation:  0.4169858587349772
Confusion Matrix : 
 [[359  94]
 [173 280]]
Accuracy on test set:    0.7052980132450332
Sensitivity:    0.6181015452538632 	 Specificity:    0.7924944812362031
Precision  : 0.7486631016042781
Area Under Curve:    0.7052980132450332
(906, 1536) (906,)
(5245, 1536) (457, 1536)
(208830, 1536) (22733, 1536)
Training Dataset
(10490, 1536) (10490,)




Validation Dataset
(914, 1536) (914,)
Independent Testing Matthews Correlation:  0.2766924798828788
Confusion Matrix : 
 [[343 114]
 [220 237]]
Accuracy on test set:    0.6345733041575492
Sensitivity:    0.5185995623632386 	 Specificity:    0.75054704595186
Precision  : 0.6752136752136753
Area Under Curve:    0.6345733041575492
(914, 1536) (914,)
(5073, 1536) (629, 1536)
(207198, 1536) (24365, 1536)
Training Dataset
(10146, 1536) (10146,)




Valida

(4917, 1536) (794, 1536)
(206378, 1536) (24288, 1536)
Training Dataset
(9834, 1536) (9834,)




Validation Dataset
(1588, 1536) (1588,)
Independent Testing Matthews Correlation:  0.40616568643462886
Confusion Matrix : 
 [[580 214]
 [258 536]]
Accuracy on test set:    0.7027707808564232
Sensitivity:    0.6750629722921915 	 Specificity:    0.7304785894206549
Precision  : 0.7146666666666667
Area Under Curve:    0.7027707808564232
(1588, 1536) (1588,)
(5291, 1536) (420, 1536)
(205715, 1536) (24951, 1536)
Training Dataset
(10582, 1536) (10582,)




Validation Dataset
(840, 1536) (840,)
Independent Testing Matthews Correlation:  0.3579657718000913
Confusion Matrix : 
 [[338  82]
 [193 227]]
Accuracy on test set:    0.6726190476190477
Sensitivity:    0.5404761904761904 	 Specificity:    0.8047619047619048
Precision  : 0.7346278317152104
Area Under Curve:    0.6726190476190476
(840, 1536) (840,)
(5161, 1536) (550, 1536)
(207656, 1536) (23010, 1536)
Training Dataset
(10322, 1536) (10322,)




V

(5013, 1536) (692, 1536)
(208973, 1536) (22361, 1536)
Training Dataset
(10026, 1536) (10026,)




Validation Dataset
(1384, 1536) (1384,)
Independent Testing Matthews Correlation:  0.40369587619954106
Confusion Matrix : 
 [[503 189]
 [224 468]]
Accuracy on test set:    0.7015895953757225
Sensitivity:    0.6763005780346821 	 Specificity:    0.726878612716763
Precision  : 0.7123287671232876
Area Under Curve:    0.7015895953757226
(1384, 1536) (1384,)
(5123, 1536) (582, 1536)
(208081, 1536) (23253, 1536)
Training Dataset
(10246, 1536) (10246,)




Validation Dataset
(1164, 1536) (1164,)
Independent Testing Matthews Correlation:  0.3786535610435059
Confusion Matrix : 
 [[418 164]
 [198 384]]
Accuracy on test set:    0.6890034364261168
Sensitivity:    0.6597938144329897 	 Specificity:    0.718213058419244
Precision  : 0.7007299270072993
Area Under Curve:    0.6890034364261168
(1164, 1536) (1164,)
(5237, 1536) (468, 1536)
(210728, 1536) (20606, 1536)
Training Dataset
(10474, 1536) (10474,)

